In [2]:
import pandas as pd
from selenium import webdriver

## Selenium
- https://www.seleniumhq.org
- 셀레니움의 기본 사용법
- 셀레니움을 이용한 파일 업로드
    - google vision api 웹페이지에 이미지 파일을 업로드, 결과 데이터를 수집
- 웹데이터 스크래핑
    - gmarket 페이지 베스트셀러 200 상품 수집
    - 네이버 본인 계정의 메일 수집 (시간되면 나중에)
    - 
- 셀레니움을 이용한 자동화
    - 특정 워드프레스 사이트의 게시물에 댓글 자동으로 달기
- 이미지 스크린샷
    - youtube 영상 이미지를 크롤링
    - pillow : python에서 이미지 처리에 관련된 패키지

In [214]:
# 브라우저 열기
driver = webdriver.Chrome() #크롬 브라우저 객체

In [215]:
driver.get("http://www.naver.com")

In [216]:
# 윈도우 사이즈 조절
driver.set_window_size(800,600)

In [217]:
# 브라우저의 스크롤 위치 조절
script = "window.scrollTo(400,200);" # Java Script
driver.execute_script(script)

In [218]:
# 윈도우 객체 저장
main_window = driver.current_window_handle
main_window

'CDwindow-C843E189A978842EE5DA4EDBA3EE9879'

In [219]:
# 윈도우 탭 열기
script = "window.open('http://www.google.com');"
driver.execute_script(script)

In [220]:
# 현재 열려있는 윈도우 객체 확인
windows = driver.window_handles
windows

['CDwindow-C843E189A978842EE5DA4EDBA3EE9879',
 'CDwindow-5E7B1303B28574D12558E43B65F5F38F']

In [222]:
# 현재 윈도우 핸들 변경
driver.switch_to_window(windows[1])
driver.current_window_handle

'CDwindow-5E7B1303B28574D12558E43B65F5F38F'

In [223]:
# 페이지 새로고침
script = "location.reload();"
driver.execute_script(script)

In [227]:
# alert 메시지 처리
script = "alert('selenium test');"
driver.execute_script(script)

In [228]:
alert = driver.switch_to_alert()
alert.text

'selenium test'

In [230]:
# confirm 메시지 처리
script = "confirm('dss8')"
driver.execute_script(script)

In [231]:
confirm = driver.switch_to_alert()
confirm.text

'dss8'

In [232]:
confirm.accept() # 확인 버튼 클릭
# confirm.dismiss() # 취소 버튼 클릭

In [234]:
# 현재 윈도우 핸들 변경
driver.switch_to_window(windows[0])
driver.current_window_handle

'CDwindow-C843E189A978842EE5DA4EDBA3EE9879'

In [235]:
# input 창에 문자열 입력하기
driver.find_element_by_css_selector("#query").send_keys("패스트 캠퍼스")

In [236]:
# 클릭 이벤트 발생
driver.find_element_by_css_selector("#search_btn").click()

In [237]:
# 창닫기
driver.close()

In [238]:
# 브라우저 닫기
driver.quit()

### 파일 업로드
- google vision api 페이지에 이미지 파일을 업로드
- iframe으로 이동

In [56]:
# 구글 비전 api 페이지로 이동
url = "https://cloud.google.com/vision/"
driver = webdriver.Chrome()
driver.get(url)

In [63]:
# iframe으로 이동
iframe = driver.find_element_by_css_selector("#vision_demo_section > iframe")
driver.switch_to_frame(iframe)

In [58]:
# 메인 프레임으로 돌아가기
driver.switch_to_default_content()

In [64]:
# 파일 업로드
# 파일 경로는 절대경로로 작성해야한다.
file_path = "/Users/sonwoncheol/Downloads/dog.png"
driver.find_element_by_css_selector("#input").send_keys(file_path)

In [65]:
# 웹 데이터 가져오기
driver.find_element_by_css_selector("#webDetection").click()

In [68]:
# find_element 가 아니라 find_elements
web_results = driver.find_elements_by_css_selector\
("#card .container > .row")[:15]
len(web_results)

15

In [69]:
for web_result in web_results:
    print(web_result.find_element_by_css_selector(".name").text)
    print(web_result.find_element_by_css_selector(".score").text)    

French Bulldog
1.28992
Boston Terrier
1.27957
Bulldog
1.27812
Bull Terrier
1.06631
Soft-coated Wheaten Terrier
0.9948
Puppy
0.9783
Border Collie
0.7819
Terrier
0.6349
Pet
0.6142
Brindle
0.5551
Dog breed
0.4963
Breed
0.4244
Veterinarian
0.4235
Black Brindle
0.4078
Dog
0


In [70]:
# 텍스트 데이터 가져오기
driver.find_element_by_css_selector("#fullTextAnnotation").click()

In [71]:
document_results = driver.find_elements_by_css_selector(".page > .container.style-scope.vs-document > fieldset")
len(document_results)

2

In [86]:
sentences = []
for document_result in document_results:
    words = document_result.find_elements_by_css_selector('.container .word')
#     print(len(words))
    words_list = []
    for word in words:
        word = word.text.replace(" ","")
        words_list.append(word)
#     print(sentence_list)
    sentences.append(" ".join(words_list))
sentences

['In the dog house', "' 인기를 잃다 , 면목을 잃다"]

## 스크래핑
- gmarket 베스트 셀러 데이터를 수집

In [199]:
url="http://corners.gmarket.co.kr/Bestsellers"
driver = webdriver.Chrome()
driver.get(url)

In [200]:
items = driver.find_elements_by_css_selector(".best-list:nth-child(3) > ul >li")
len(items)

200

In [201]:
dict_list = []
for item in items:
    dict_list.append({
        "title":item.find_element_by_css_selector('.itemname').text,
        "s-price":item.find_element_by_css_selector('.s-price').text\
        .split(" ")[0].replace(",","")[:-1],
        "o-price":item.find_element_by_css_selector('.o-price').text\
        .split(" ")[0].replace(",","")[:-1],
        "link":item.find_element_by_css_selector('.itemname').get_attribute("href")
    })
# dict_list
df = pd.DataFrame(dict_list)
df

,link,o-price,s-price,title
0,http://item.gmarket.co.kr/Item?goodscode=14301...,39000,28470,[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스
1,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈
2,http://item.gmarket.co.kr/Item?goodscode=63617...,30000,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수
3,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정)
4,http://item.gmarket.co.kr/Item?goodscode=41037...,24000,4900,[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70
5,http://item.gmarket.co.kr/Item?goodscode=87359...,30000,13900,[동원덴마크우유]동원 인포켓치즈 20g x20개+(저키x8개 or 치즈디퍼즈)
6,http://item.gmarket.co.kr/Item?goodscode=91202...,27100,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반
7,http://item.gmarket.co.kr/Item?goodscode=14322...,,22670,AS가능아디다스국내매장정품스탠다드반팔티셔츠CW0711
8,http://item.gmarket.co.kr/Item?goodscode=11767...,32900,9900,언니드랑 블라우스/원피스/팬츠/스커트/린넨
9,http://item.gmarket.co.kr/Item?goodscode=14365...,39900,32900,[하기스]2018년형 에어솔솔 썸머 팬티 / 밴드 기저귀 2팩


In [202]:
is_nulldata = df["o-price"] == ""
df["o-price"][is_nulldata]

7      
24     
44     
49     
80     
98     
107    
117    
124    
147    
163    
167    
179    
182    
188    
197    
198    
Name: o-price, dtype: object

In [203]:
df["s-price"][is_nulldata]

7      22670
24     53000
44     46900
49     96000
80     11900
98     39800
107    31270
117     7000
124    18900
147    84900
163    28900
167    38500
179    19900
182    29800
188    39900
197    14900
198    23900
Name: s-price, dtype: object

In [204]:
df["o-price"][is_nulldata] = df["s-price"][is_nulldata]

In [205]:
df

,link,o-price,s-price,title
0,http://item.gmarket.co.kr/Item?goodscode=14301...,39000,28470,[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스
1,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈
2,http://item.gmarket.co.kr/Item?goodscode=63617...,30000,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수
3,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정)
4,http://item.gmarket.co.kr/Item?goodscode=41037...,24000,4900,[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70
5,http://item.gmarket.co.kr/Item?goodscode=87359...,30000,13900,[동원덴마크우유]동원 인포켓치즈 20g x20개+(저키x8개 or 치즈디퍼즈)
6,http://item.gmarket.co.kr/Item?goodscode=91202...,27100,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반
7,http://item.gmarket.co.kr/Item?goodscode=14322...,22670,22670,AS가능아디다스국내매장정품스탠다드반팔티셔츠CW0711
8,http://item.gmarket.co.kr/Item?goodscode=11767...,32900,9900,언니드랑 블라우스/원피스/팬츠/스커트/린넨
9,http://item.gmarket.co.kr/Item?goodscode=14365...,39900,32900,[하기스]2018년형 에어솔솔 썸머 팬티 / 밴드 기저귀 2팩


In [206]:
df1 = df

In [207]:
df1.shape

(200, 4)

In [208]:
df1.head()

,link,o-price,s-price,title
0,http://item.gmarket.co.kr/Item?goodscode=14301...,39000,28470,[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스
1,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈
2,http://item.gmarket.co.kr/Item?goodscode=63617...,30000,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수
3,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정)
4,http://item.gmarket.co.kr/Item?goodscode=41037...,24000,4900,[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70


In [209]:
# 할인율 컬럼 추가하기 (소수점 없이 (ex: 27%))
# object로 되어 있는 series를 int로 형변환하여 할인율을 구합니다.
df1["discount_rate"] = round(1 - df1["s-price"].astype("int") / df1["o-price"].astype("int"), 2) * 100
# 소수점 (.0) 제거
df1["discount_rate"] = df1["discount_rate"].astype("int")
# % 추가
df1["discount_rate"] = df1["discount_rate"].astype("str")
df1["discount_rate"] += "%"
df1

,link,o-price,s-price,title,discount_rate
0,http://item.gmarket.co.kr/Item?goodscode=14301...,39000,28470,[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스,27%
1,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈,70%
2,http://item.gmarket.co.kr/Item?goodscode=63617...,30000,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수,56%
3,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정),36%
4,http://item.gmarket.co.kr/Item?goodscode=41037...,24000,4900,[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70,80%
5,http://item.gmarket.co.kr/Item?goodscode=87359...,30000,13900,[동원덴마크우유]동원 인포켓치즈 20g x20개+(저키x8개 or 치즈디퍼즈),54%
6,http://item.gmarket.co.kr/Item?goodscode=91202...,27100,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반,38%
7,http://item.gmarket.co.kr/Item?goodscode=14322...,22670,22670,AS가능아디다스국내매장정품스탠다드반팔티셔츠CW0711,0%
8,http://item.gmarket.co.kr/Item?goodscode=11767...,32900,9900,언니드랑 블라우스/원피스/팬츠/스커트/린넨,70%
9,http://item.gmarket.co.kr/Item?goodscode=14365...,39900,32900,[하기스]2018년형 에어솔솔 썸머 팬티 / 밴드 기저귀 2팩,18%


In [210]:
# 할인율 컬럼 추가하기 (소수점 없이 (ex: 27%))
# object로 되어 있는 series를 int로 형변환하여 할인율을 구합니다.
df1["discount_rate"] = round(1 - df1["s-price"].astype("int") / df1["o-price"].astype("int"), 2) * 100
# 소수점 (.0) 제거
df1["discount_rate"] = df1["discount_rate"].astype("int")
# % 추가
df1["discount_rate"] = df1["discount_rate"].astype("str")
df1["discount_rate"] += "%"
df1

,link,o-price,s-price,title,discount_rate
0,http://item.gmarket.co.kr/Item?goodscode=14301...,39000,28470,[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스,27%
1,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈,70%
2,http://item.gmarket.co.kr/Item?goodscode=63617...,30000,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수,56%
3,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정),36%
4,http://item.gmarket.co.kr/Item?goodscode=41037...,24000,4900,[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70,80%
5,http://item.gmarket.co.kr/Item?goodscode=87359...,30000,13900,[동원덴마크우유]동원 인포켓치즈 20g x20개+(저키x8개 or 치즈디퍼즈),54%
6,http://item.gmarket.co.kr/Item?goodscode=91202...,27100,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반,38%
7,http://item.gmarket.co.kr/Item?goodscode=14322...,22670,22670,AS가능아디다스국내매장정품스탠다드반팔티셔츠CW0711,0%
8,http://item.gmarket.co.kr/Item?goodscode=11767...,32900,9900,언니드랑 블라우스/원피스/팬츠/스커트/린넨,70%
9,http://item.gmarket.co.kr/Item?goodscode=14365...,39900,32900,[하기스]2018년형 에어솔솔 썸머 팬티 / 밴드 기저귀 2팩,18%


In [211]:
# 할인율 컬럼 추가하기 (소수점 없이 (27%))
# object로 되어 있는 series를 int로 형변환하여 할인율을 구한다,.
df1["discount_rate"] = round(1 - df1["s-price"].astype("int") \
                             / df1["o-price"].astype("int"), 2) * 100
# 소수점 (.0) 제거
df1["discount_rate"] = df1["discount_rate"].astype("int")
# % 추가
df1["discount_rate"] = df1["discount_rate"].astype("str")
df1["discount_rate"] += "%"
df1

,link,o-price,s-price,title,discount_rate
0,http://item.gmarket.co.kr/Item?goodscode=14301...,39000,28470,[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스,27%
1,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈,70%
2,http://item.gmarket.co.kr/Item?goodscode=63617...,30000,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수,56%
3,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정),36%
4,http://item.gmarket.co.kr/Item?goodscode=41037...,24000,4900,[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70,80%
5,http://item.gmarket.co.kr/Item?goodscode=87359...,30000,13900,[동원덴마크우유]동원 인포켓치즈 20g x20개+(저키x8개 or 치즈디퍼즈),54%
6,http://item.gmarket.co.kr/Item?goodscode=91202...,27100,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반,38%
7,http://item.gmarket.co.kr/Item?goodscode=14322...,22670,22670,AS가능아디다스국내매장정품스탠다드반팔티셔츠CW0711,0%
8,http://item.gmarket.co.kr/Item?goodscode=11767...,32900,9900,언니드랑 블라우스/원피스/팬츠/스커트/린넨,70%
9,http://item.gmarket.co.kr/Item?goodscode=14365...,39900,32900,[하기스]2018년형 에어솔솔 썸머 팬티 / 밴드 기저귀 2팩,18%


In [87]:
df.head()

,link,o-price,s-price,title
0,http://item.gmarket.co.kr/Item?goodscode=13005...,20000,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정)
1,http://item.gmarket.co.kr/Item?goodscode=94765...,65900,19900,티셔츠5장9900 원피스/티셔츠/린넨팬츠 세트
2,http://item.gmarket.co.kr/Item?goodscode=14413...,100000,49900,[풍년]PN풍년 럭키 반값박스 (압력솥+프라이팬+냄비+사은품)
3,http://item.gmarket.co.kr/Item?goodscode=91202...,27100,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반
4,http://item.gmarket.co.kr/Item?goodscode=62724...,19500,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈


In [88]:
driver.quit()

### 워드프레스 사이트에 댓글 쓰기
- 2개의 계정, 2개의 메시지를 2개의 게시글에 자동으로 작성
- https://radajin661456067.wordpress.com/

In [180]:
url = "https://radajin661456067.wordpress.com/"
driver = webdriver.Chrome()
driver.get(url)

In [182]:
# 게시물 링크 가져오기
links = []
articles = driver.find_elements_by_css_selector("#main > article")
for article in articles:
    links.append(
        article.find_element_by_css_selector(".entry-title > a")\
        .get_attribute("href")
    )
links

['https://radajin661456067.wordpress.com/2018/06/21/%ed%85%8c%ec%8a%a4%ed%8a%b8/',
 'https://radajin661456067.wordpress.com/2018/06/21/%ec%b2%ab-%eb%b2%88%ec%a7%b8-%eb%b8%94%eb%a1%9c%ea%b7%b8-%ea%b8%80/']

In [183]:
# 링크로 이동
driver.get(links[1])

In [184]:
# 스크롤 내리기
script = "window.scrollTo(0,1000);"
driver.execute_script(script)

In [185]:
# 댓글창 클릭
driver.find_element_by_css_selector("#comment-form-comment").click()

In [186]:
# 댓글 내용 작성
driver.find_element_by_css_selector("#comment").send_keys("hello hello 데사스 화이팅")

In [187]:
# 스크롤 내리기
script = "window.scrollTo(0,1000);"
driver.execute_script(script)

In [188]:
# 이메일, 이름 작성하기
import time
time.sleep(3)
driver.find_element_by_css_selector("#email").clear()
driver.find_element_by_css_selector("#email").send_keys("fwdfwer@naver.com")
time.sleep(3)
driver.find_element_by_css_selector("#author").clear()
driver.find_element_by_css_selector("#author").send_keys("i am fwdfwer")

In [189]:
# 댓글 작성 버튼 클릭
driver.find_element_by_css_selector("#comment-submit").click()

In [176]:
driver.quit()

In [190]:
links

['https://radajin661456067.wordpress.com/2018/06/21/%ed%85%8c%ec%8a%a4%ed%8a%b8/',
 'https://radajin661456067.wordpress.com/2018/06/21/%ec%b2%ab-%eb%b2%88%ec%a7%b8-%eb%b8%94%eb%a1%9c%ea%b7%b8-%ea%b8%80/']

In [191]:
def write_comment(driver, link, msg, email, name):
    driver.get(link)
    script = "window.scrollTo(0,1000);"
    driver.execute_script(script)
    time.sleep(2)
    driver.find_element_by_css_selector("#comment-form-comment").click()
    driver.find_element_by_css_selector("#comment").send_keys(msg)
    script = "window.scrollTo(0,1000);"
    driver.execute_script(script)
    time.sleep(1)
    driver.find_element_by_css_selector("#email").clear()
    driver.find_element_by_css_selector("#email").send_keys("fwdfwer@naver.com")
    time.sleep(1)
    driver.find_element_by_css_selector("#author").clear()
    driver.find_element_by_css_selector("#author").send_keys("i am fwdfwer")
    script = "window.scrollTo(0,1000);"
    driver.execute_script(script)
    time.sleep(1)
    driver.find_element_by_css_selector("#comment-submit").click()    

In [192]:
write_comment(driver, links[1], "대~한민국!!","red-devil@gmail.com","red-devil")

In [193]:
driver.quit()

In [144]:
users=[
    {"email":"red-devil@naver.com","name":"red-devil"},
    {"email":"red-devil@gmail.com","name":"red-devil"},
]
msgs=["한국힘내라","멕시코이기자"]

driver = webdriver.Chrome()

for user in users:
    for msg in msgs:
        for link in links:
            print(user,msg, end=" - ")
            try:
                write_comment(driver, link, msg, user["email"], user["name"])
                print("성공")
            except:
                print("실패")
            time.sleep(2)

{'email': 'red-devil@naver.com', 'name': 'red-devil'} 한국힘내라 - 성공
{'email': 'red-devil@naver.com', 'name': 'red-devil'} 한국힘내라 - 성공
{'email': 'red-devil@naver.com', 'name': 'red-devil'} 멕시코이기자 - 성공
{'email': 'red-devil@naver.com', 'name': 'red-devil'} 멕시코이기자 - 실패
{'email': 'red-devil@gmail.com', 'name': 'red-devil'} 한국힘내라 - 성공
{'email': 'red-devil@gmail.com', 'name': 'red-devil'} 한국힘내라 - 실패
{'email': 'red-devil@gmail.com', 'name': 'red-devil'} 멕시코이기자 - 성공
{'email': 'red-devil@gmail.com', 'name': 'red-devil'} 멕시코이기자 - 실패


In [147]:
driver.quit()

## 네이버 메일 크롤링
- 네이버 페이지로 이동
- 아이디 패스워드로 로그인
- 전체 메일수, 안 읽은 메일수
- 메일 제목과 요약을 데이터 프레임으로 만들기

In [146]:
main_url = "http://www.naver.com"
mail_url = "http://mail.naver.com"

In [148]:
driver = webdriver.Chrome()
driver.get(main_url)

In [149]:
import pickle

In [150]:
with open('pw.plk','wb') as f:
    pickle.dump("godusj13@#",f)

In [151]:
pw = pickle.load(open("pw.plk","rb"))

In [152]:
# 아이디 패스워드 입력
driver.find_element_by_css_selector("#id").send_keys("1chuu")
driver.find_element_by_css_selector("#pw").send_keys(pw)

In [155]:
# 로그인 버튼 클릭
driver.find_element_by_css_selector("span.btn_login > input[type=submit]").click()

In [156]:
# 메일 체이지로 이동
driver.get(mail_url)

In [157]:
# 전체 메일 갯수 가져오기
total_mail_count = driver.find_element_by_css_selector("#headTotalNum").text
total_mail_count

'5355'

In [162]:
# 안읽은 메일 갯수 가져오기
unread_mail_count = driver.find_element_by_css_selector("#headUnreadNum").text
unread_mail_count

'5233'

In [163]:
# 메일 데이터 가져오기 - 제목, 보낸사람, 메일링크
mails = driver.find_elements_by_css_selector(".mailList > li")
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for mail in mails:
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a")\
    .get_attribute("href")
    df.loc[len(df)]={"제목":title, "보낸사람":name, "메일링크":link}
#     print(name, title, link)
df

,제목,보낸사람,메일링크
0,(광고) 창업을 생각중인가요? 성공창업을 위한 무점포1인창업과 창업정보 및 다양한 ...,창업신문,https://mail.naver.com/read/popup/?nMailId=5657
1,"(광고) 【영어꿀팁】 오늘 저녁은 내가 살게! ""It's on me!""",민병철유폰,https://mail.naver.com/read/popup/?nMailId=5656
2,"(광고) 프라우드비뇨기과, 남성수술 49%양심할인 혜택 선착순 100명 무료 신청",Namepr,https://mail.naver.com/read/popup/?nMailId=5655
3,"(광고) 양말 20켤레 --> 4,900원",위메프,https://mail.naver.com/read/popup/?nMailId=5654
4,(광고) 끼니 거르지마~ 볶음밥,위메프,https://mail.naver.com/read/popup/?nMailId=5653
5,Insider deals for you - great deals for your t...,Agoda's Best H..,https://mail.naver.com/read/popup/?nMailId=5652
6,[유학네트] 2018년도 개인정보 이용내역 안내를 드립니다.,유학네트,https://mail.naver.com/read/popup/?nMailId=5651
7,"(광고) APP알림 켜두시고 3,000P 받으세요!",위메프,https://mail.naver.com/read/popup/?nMailId=5650
8,(광고) 양말 20켤레 초특가!,위메프,https://mail.naver.com/read/popup/?nMailId=5649
9,(광고)[금융권 취업 정보] 금융권 취준생을 위한 금융멘토 맹재원의 금융권 박살내기,금융권 취업정보,https://mail.naver.com/read/popup/?nMailId=5648


In [164]:
driver.quit()

## 이미지 스크린샷
- youtube 영상을 이미지로 스크린샷을 만들고
- 영상 있는 부분의 위치를 파악
- 영상이 있는 부분을 pillow를 이용해서 자르기
- 영상을 저장해서 위해서 실습했던 google vision api에 넣어서 결과를 확인